# RAG Evaluation Notebook

Этот ноутбук:
1. Загружает данные из Weaviate
2. Генерирует вопросы к описаниям событий через OpenAI LLM
3. Сохраняет датасет с вопросами в `testing_data/`
4. Запускает RAG систему с вопросами
5. Вычисляет метрики Recall@K, Precision@K, MRR, NDCG@K и т.д.

## Инструкции по запуску

```bash
# Запустите Docker с Weaviate
docker-compose up -d weaviate contextionary
```


## 1. Настройка и импорты


In [1]:
import sys
import os
from pathlib import Path

# Добавляем корневую директорию проекта в путь
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")


Project root: /Users/andreisuhov/Desktop/llm/Journey_agent


In [2]:
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional, Set
from dataclasses import dataclass
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)

# Загружаем переменные окружения из .env
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Импорты из проекта
from src.vdb import (
    get_weaviate_client,
    COLLECTION_NAME,
    WEAVIATE_URL,
    EventRetriever,
)
from src.launch_pipeline import launch_pipeline
from src.models.event import Event

# OpenAI
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"WEAVIATE_URL: {WEAVIATE_URL}")
print(f"COLLECTION_NAME: {COLLECTION_NAME}")
print(f"OpenAI API Key: {'✓ загружен' if OPENAI_API_KEY else '✗ НЕ НАЙДЕН'}")


/Users/andreisuhov/miniconda3/envs/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WEAVIATE_URL: http://localhost:8080
COLLECTION_NAME: Events
OpenAI API Key: ✓ загружен


## 2. Запуск launch_pipeline и загрузка данных


In [3]:
# Запускаем launch_pipeline - ожидание Weaviate, создание коллекции, загрузка данных
launch_pipeline()

print("\n✓ Pipeline выполнен")


Ожидание подключения к Weaviate по адресу: http://localhost:8080
Таймаут: 60 секунд
✓ Weaviate подключен успешно за 0.43 секунд
ℹ️  Коллекция 'Events' уже существует
   Количество записей в коллекции: 654
   Для пересоздания с новой конфигурацией используйте: python /Users/andreisuhov/Desktop/llm/Journey_agent/src/vdb/utils/add_events.py --recreate
Parsing file: /Users/andreisuhov/Desktop/llm/Journey_agent/data/raw_data/real_events_data/events_pydantic_msk_20251212_130919.json
Loaded 570 events
Loading events to Weaviate
📤 Загрузка 570 событий в Weaviate...

Загрузка завершена!
   Успешно: 0
   Ошибок: 0
   Пропущено: 570
Parsing file: /Users/andreisuhov/Desktop/llm/Journey_agent/data/raw_data/real_events_data/events_pydantic_spb_20251212_131151.json
Loaded 518 events
Loading events to Weaviate
📤 Загрузка 518 событий в Weaviate...

Загрузка завершена!
   Успешно: 0
   Ошибок: 0
   Пропущено: 518

✓ Pipeline выполнен


In [6]:
import weaviate.classes as wvc

def get_all_events_from_db(limit: int = 1000) -> List[Dict[str, Any]]:
    """Получает все события из Weaviate."""
    client = get_weaviate_client()
    events_with_ids = []
    
    try:
        collection = client.collections.get(COLLECTION_NAME)
        result = collection.query.fetch_objects(limit=limit)
        
        for idx, obj in enumerate(result.objects):
            event_data = {
                "id": idx,
                "description": obj.properties.get("description", ""),
                "title": obj.properties.get("title", ""),
                "location": obj.properties.get("location", ""),
                "date": obj.properties.get("date", ""),
                "tags": obj.properties.get("tags", []),
            }
            events_with_ids.append(event_data)
    finally:
        client.close()
    
    return events_with_ids

# Получаем все события
all_db_events = get_all_events_from_db()
df_events = pd.DataFrame(all_db_events)

print(f"Всего событий в базе: {len(df_events)}")
print(f"\nКолонки: {df_events.columns.tolist()}")
df_events[["id", "title", "description"]].head(5)


Всего событий в базе: 654

Колонки: ['id', 'description', 'title', 'location', 'date', 'tags']


,id,title,description
0,0,базовый курс рисования в студии «Рисуй»,"Карандашные зарисовки, масляные натюрморты, по..."
1,1,постоянная экспозиция «Советский фарфор»,"Изделия из фарфора и фаянса, сравнимые с насто..."
2,2,постоянная экспозиция Петропавловской крепости...,Выставка дает ответы на многие вопросы: какие ...
3,3,коллекция Эрмитажа «Русский народный костюм»,Коллекция включает в себя более тысячи предмет...
4,4,интерактивная выставка «Капибарабууум»,Новая выставка-игра «Москвариума» посвящена ка...


## 3. Генерация вопросов через OpenAI LLM


In [8]:
client_openai = OpenAI(api_key=OPENAI_API_KEY)

def generate_questions(description: str, title: str = "") -> List[str]:
    """
    Генерирует 2 вопроса к описанию события через OpenAI.
    
    Args:
        description: Описание события
        title: Название события (опционально)
    
    Returns:
        Список из 2 вопросов
    """
    if not description or len(description.strip()) < 10:
        return []
    
    prompt = f"""На основе описания мероприятия сгенерируй ровно 2 естественных вопроса, 
которые пользователь мог бы задать, чтобы найти это мероприятие.
Вопросы должны быть разнообразными и отражать суть события.
Пользователь пишет запросы типа: хочу сходить на какое то мероприятие готической музыки и потом покушать
Тебе надо придумать такие запросы, на которые раг система вывела бы это событие, которое может относиться например лишь к части запроса
Название: {title}
Описание: {description[:500]}

Ответь ТОЛЬКО двумя вопросами, каждый на новой строке, без нумерации и лишнего текста."""

    # try:
    response = client_openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=550,
    )
    
    text = response.choices[0].message.content.strip()
    questions = [q.strip() for q in text.split("\n") if q.strip()]
    return questions  # Возвращаем максимум 2 вопроса
    
    # except Exception as e:
    #     print(f"Ошибка генерации: {e}")
    #     return []

# Тест генерации
test_desc = df_events.iloc[0]["description"]
test_title = df_events.iloc[0]["title"]
print(f"Тест для: {test_title[:50]}...")
print(f"Описание: {test_desc[:100]}...")
print(f"\nСгенерированные вопросы:")
for q in generate_questions(test_desc, test_title):
    print(f"  - {q}")


Тест для: базовый курс рисования в студии «Рисуй»...
Описание: Карандашные зарисовки, масляные натюрморты, портрет углём и реалистичные пейзажи — за 16 занятий вы ...

Сгенерированные вопросы:
  - Где можно пройти курс по основам живописи и рисунка?
  - Какие студии предлагают занятия по рисованию маслом и углём?


In [ ]:
# Генерируем вопросы для всех событий (или подвыборки для экономии)
# Ограничим количество для экономии токенов
SAMPLE_SIZE = 50  # Измените на None для обработки всех

if SAMPLE_SIZE:
    df_sample = df_events.sample(n=min(SAMPLE_SIZE, len(df_events)), random_state=42).reset_index(drop=True)
    print(f"Используем выборку из {len(df_sample)} событий")
else:
    df_sample = df_events.copy()
    print(f"Обрабатываем все {len(df_sample)} событий")

# Генерируем вопросы
all_queries = []

for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Генерация вопросов"):
    questions = generate_questions(row["description"], row["title"])
    
    for q in questions:
        all_queries.append({
            "id": row["id"],
            "title": row["title"],
            "description": row["description"],
            "query": q,
        })

df_queries = pd.DataFrame(all_queries)
print(f"\n✓ Сгенерировано {len(df_queries)} вопросов для {len(df_sample)} событий")
df_queries.head(10)


Используем выборку из 50 событий


Генерация вопросов: 100%|██████████| 50/50 [01:09<00:00,  1.39s/it]


✓ Сгенерировано 100 вопросов для 50 событий


,id,title,description,query
0,630,симфоническое Rammstein-шоу от Amadeus Concerts,Хиты легендарной немецкой команды ещё никогда ...,Где можно услышать симфоническое исполнение пе...
1,630,симфоническое Rammstein-шоу от Amadeus Concerts,Хиты легендарной немецкой команды ещё никогда ...,Какое музыкальное шоу предлагает необычные ара...
2,271,Стендап для детей: семейное комедийное шоу вых...,Юмористический концерт для детей в стенах наст...,Где можно весело провести выходной с детьми на...
3,271,Стендап для детей: семейное комедийное шоу вых...,Юмористический концерт для детей в стенах наст...,Есть ли в городе стендап-клубы с выступлениями...
4,135,"мастер-классы по керамике и лепке, занятия за ...",Проведите время весело и увлекательно — почувс...,Где в Москве можно посетить мастер-класс по ке...
5,135,"мастер-классы по керамике и лепке, занятия за ...",Проведите время весело и увлекательно — почувс...,"Есть ли в городе студии, где можно попробовать..."
6,483,концерт «Аватар. Музыкальное шоу в Планетарии 1»,Уникальное шоу от Amadeus Concerts — полное по...,Где можно послушать музыку из фильма «Аватар» ...
7,483,концерт «Аватар. Музыкальное шоу в Планетарии 1»,Уникальное шоу от Amadeus Concerts — полное по...,Какие концерты проходят в Планетарии 1 в ближа...
8,90,постоянная экспозиция Музея кукол «Петербургск...,"Ироничный, но добрый художественный проект, кр...",Где можно увидеть выставку с куклами известных...
9,90,постоянная экспозиция Музея кукол «Петербургск...,"Ироничный, но добрый художественный проект, кр...","Какие художественные проекты можно посетить, ч..."


## 4. Сохранение датасета в testing_data


In [36]:
# Сохраняем датасет
testing_data_dir = Path(project_root / "data/testing_data")
testing_data_dir.mkdir(exist_ok=True)

output_file = testing_data_dir / "rag_test_queries.csv"
df_queries.to_csv(output_file, index=False, encoding="utf-8")

# Также сохраняем в JSON для удобства
output_json = testing_data_dir / "rag_test_queries.json"
df_queries.to_json(output_json, orient="records", force_ascii=False, indent=2)

print(f"✓ Датасет сохранён:")
print(f"  - CSV: {output_file}")
print(f"  - JSON: {output_json}")
print(f"  - Записей: {len(df_queries)}")


✓ Датасет сохранён:
  - CSV: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/rag_test_queries.csv
  - JSON: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/rag_test_queries.json
  - Записей: 100


In [35]:
!pwd

/Users/andreisuhov/Desktop/llm/Journey_agent/tests


## 5. Загрузка датасета (если уже сгенерирован)


In [4]:
# Если датасет уже существует, можно загрузить его
testing_data_dir = Path(project_root/ "data/testing_data")
saved_file = testing_data_dir / "rag_test_queries.csv"

if saved_file.exists():
    df_queries = pd.read_csv(saved_file)
    print(f"✓ Загружен существующий датасет: {len(df_queries)} записей")
else:
    print("⚠️ Датасет не найден. Запустите ячейки генерации выше.")

df_queries.head(5)


✓ Загружен существующий датасет: 100 записей


,id,title,description,query
0,630,симфоническое Rammstein-шоу от Amadeus Concerts,Хиты легендарной немецкой команды ещё никогда ...,Где можно услышать симфоническое исполнение пе...
1,630,симфоническое Rammstein-шоу от Amadeus Concerts,Хиты легендарной немецкой команды ещё никогда ...,Какое музыкальное шоу предлагает необычные ара...
2,271,Стендап для детей: семейное комедийное шоу вых...,Юмористический концерт для детей в стенах наст...,Где можно весело провести выходной с детьми на...
3,271,Стендап для детей: семейное комедийное шоу вых...,Юмористический концерт для детей в стенах наст...,Есть ли в городе стендап-клубы с выступлениями...
4,135,"мастер-классы по керамике и лепке, занятия за ...",Проведите время весело и увлекательно — почувс...,Где в Москве можно посетить мастер-класс по ке...


## 6. Запуск RAG и сбор результатов


In [7]:
# Создаём маппинг description -> id для матчинга результатов
description_to_id = {row["description"]: row["id"] for _, row in df_events.iterrows()}
from src.utils.journey_llm import JourneyLLM
from src.vdb import EventRetriever
from src.planner_agent.graph import PlanningGraph
from src.vdb.rag.self_rag_graph import run_self_rag
from src.utils.safety import moderate_text, SafetyLabel


llm = JourneyLLM()
retriever = EventRetriever()

def run_rag_query(query: str, limit: int = 10) -> List[Dict[str, Any]]:
    """
    Запускает RAG retriever и возвращает найденные документы.
    
    Args:
        query: Поисковый запрос
        limit: Максимум результатов
    
    Returns:
        Список найденных событий с их описаниями
    """
    retriever = EventRetriever()
    print('query: ', query)
    try:
        events = retriever.retrieve(query, limit=limit)
        print('events: ', events )
        results = []
        for event in events:
            results.append({
                "title": event.title,
                "description": event.description,
            })
        return results
    finally:
        retriever.close()

# Тест RAG
test_query = df_queries.iloc[0]["query"]
print(f"Тестовый запрос: {test_query}")
print(f"\nРезультаты RAG:")
for i, res in enumerate(run_rag_query(test_query, limit=3), 1):
    print(f"  {i}. {res['title'][:50]}...")


Тестовый запрос: Где можно услышать симфоническое исполнение песен Rammstein?

Результаты RAG:
query:  Где можно услышать симфоническое исполнение песен Rammstein?
events:  [Event(title='симфоническое Rammstein-шоу от Amadeus Concerts', owner='all', description='Хиты легендарной немецкой команды ещё никогда не звучали столь грандиозно.', tags=['all', 'concert', 'spb', 'каверы', 'концерт года', 'метал', 'концерты', 'симфонический оркестр', '16+', 'рок и рок-н-ролл', 'шоу', '16+'], source='kudago', country='Россия', location='клуб А2, просп. Медиков, д. 3, м. Петроградская', date='2022-09-09 20:00', url='https://kudago.com/spb/event/kontsert-rammstein-shou/', uuid='f4c09706-464d-5ecc-b0ad-5f98ac898ba6'), Event(title='звёздное шоу с камерным оркестром «Вселенная Ханса Циммера»', owner='all', description='Звуки органа, звёздный купол и величественная музыка кино-композитора Ханса Циммера — рецепт идеального свидания.', tags=['all', 'concert', 'spb', '6+', 'романтичные места и события', 'но

In [8]:
df_queries

,id,title,description,query
0,630,симфоническое Rammstein-шоу от Amadeus Concerts,Хиты легендарной немецкой команды ещё никогда ...,Где можно услышать симфоническое исполнение пе...
1,630,симфоническое Rammstein-шоу от Amadeus Concerts,Хиты легендарной немецкой команды ещё никогда ...,Какое музыкальное шоу предлагает необычные ара...
2,271,Стендап для детей: семейное комедийное шоу вых...,Юмористический концерт для детей в стенах наст...,Где можно весело провести выходной с детьми на...
3,271,Стендап для детей: семейное комедийное шоу вых...,Юмористический концерт для детей в стенах наст...,Есть ли в городе стендап-клубы с выступлениями...
4,135,"мастер-классы по керамике и лепке, занятия за ...",Проведите время весело и увлекательно — почувс...,Где в Москве можно посетить мастер-класс по ке...
...,...,...,...,...
95,120,акварельные мастер-классы в студии рисования Z...,Акварель открывает неограниченный простор для ...,Какие мастер-классы по рисованию проходят в Мо...
96,55,детская экскурсия-квест по Петропавловской кре...,Компания «Палитра Петербурга» приглашает ребят...,Какие есть детские экскурсии в Санкт-Петербург...
97,55,детская экскурсия-квест по Петропавловской кре...,Компания «Палитра Петербурга» приглашает ребят...,Где в Санкт-Петербурге можно поучаствовать в к...
98,319,постоянная экспозиция «Дворец во дворце» музея...,В Оперном доме представлена богатая коллекция ...,Где можно посмотреть коллекцию скульптур из му...


In [9]:
test_query = df_queries.iloc[1]["query"]
answer = df_queries.iloc[1]["description"]
print(f"Тестовый запрос: {test_query}")
print(f"Ответ: {answer}")
print(f"\nРезультаты RAG:")
ress = run_rag_query(test_query, limit=30)
for i, res in enumerate(run_rag_query(test_query, limit=30), 1):
    print(f"  {i}. {res['title'][:50]}...")

Тестовый запрос: Какое музыкальное шоу предлагает необычные аранжировки хитов немецкой группы?
Ответ: Хиты легендарной немецкой команды ещё никогда не звучали столь грандиозно.

Результаты RAG:
query:  Какое музыкальное шоу предлагает необычные аранжировки хитов немецкой группы?
events:  [Event(title='симфоническое Rammstein-шоу от Amadeus Concerts', owner='all', description='Хиты легендарной немецкой команды ещё никогда не звучали столь грандиозно.', tags=['all', 'concert', 'spb', 'каверы', 'концерт года', 'метал', 'концерты', 'симфонический оркестр', '16+', 'рок и рок-н-ролл', 'шоу', '16+'], source='kudago', country='Россия', location='клуб А2, просп. Медиков, д. 3, м. Петроградская', date='2022-09-09 20:00', url='https://kudago.com/spb/event/kontsert-rammstein-shou/', uuid='f4c09706-464d-5ecc-b0ad-5f98ac898ba6'), Event(title='концерт «Все произведения И. С. Баха для органа. Авторский цикл»', owner='all', description='Александр Фисейский представляет грандиозный цикл концертов, в кот

In [19]:
ress

[{'title': 'симфоническое Rammstein-шоу от Amadeus Concerts',
  'description': 'Хиты легендарной немецкой команды ещё никогда не звучали столь грандиозно.'},
 {'title': 'концерт «Все произведения И. С. Баха для органа. Авторский цикл»',
  'description': 'Александр Фисейский представляет грандиозный цикл концертов, в котором прозвучат все органные сочинения немецкого гения барокко.'},
 {'title': 'иммерсивное музыкально-театральное шоу «Репанутые»',
  'description': 'Забудьте всё, что вы знали о традиционных концертах. «Репанутые» — это шоу, где музыканты импровизируют на грани бунта. Здесь зрители становятся участниками действа, а виртуозные музыканты каждый раз исполняют уникальную гремучую смесь классики, джаза и рока, которую больше никто и никогда не услышит.'},
 {'title': 'иммерсивное музыкально-театральное шоу «Репанутые»',
  'description': 'Забудьте всё, что вы знали о традиционных концертах. «Репанутые» — это шоу, где музыканты импровизируют на грани бунта. Здесь зрители становя

In [19]:
# Запускаем RAG для всех вопросов и собираем результаты
K_VALUES = [1, 3, 5, 10]
MAX_RESULTS = 15  # Максимум результатов от RAG

evaluation_results = []

for idx, row in tqdm(df_queries.iterrows(), total=len(df_queries), desc="RAG оценка"):
    query = row["query"]
    ground_truth_id = row["id"]
    ground_truth_desc = row["description"]
    
    # Запускаем RAG
    rag_results = run_rag_query(query, limit=MAX_RESULTS)
    print('RAG results')
    print(rag_results)
    
    # Получаем ID найденных документов по description
    retrieved_ids = []
    for res in rag_results:
        doc_id = description_to_id.get(res["description"])
        if doc_id is not None:
            retrieved_ids.append(doc_id)

    print('retrieved_ids')
    print(retrieved_ids)
    
    evaluation_results.append({
        "query": query,
        "ground_truth_id": description_to_id[ground_truth_desc],
        "retrieved_ids": retrieved_ids,
        "num_retrieved": len(retrieved_ids),
    })

print(f"\n✓ Обработано {len(evaluation_results)} запросов")


RAG оценка:   0%|          | 0/100 [00:00<?, ?it/s]

query:  Где можно услышать симфоническое исполнение песен Rammstein?


RAG оценка:   1%|          | 1/100 [00:02<04:03,  2.46s/it]

events:  [Event(title='симфоническое Rammstein-шоу от Amadeus Concerts', owner='all', description='Хиты легендарной немецкой команды ещё никогда не звучали столь грандиозно.', tags=['all', 'concert', 'spb', 'каверы', 'концерт года', 'метал', 'концерты', 'симфонический оркестр', '16+', 'рок и рок-н-ролл', 'шоу', '16+'], source='kudago', country='Россия', location='клуб А2, просп. Медиков, д. 3, м. Петроградская', date='2022-09-09 20:00', url='https://kudago.com/spb/event/kontsert-rammstein-shou/', uuid='f4c09706-464d-5ecc-b0ad-5f98ac898ba6'), Event(title='звёздное шоу с камерным оркестром «Вселенная Ханса Циммера»', owner='all', description='Звуки органа, звёздный купол и величественная музыка кино-композитора Ханса Циммера — рецепт идеального свидания.', tags=['all', 'concert', 'spb', '6+', 'романтичные места и события', 'новое на сайте', 'космос', 'концерты', 'билеты на kudago', 'места для свиданий', 'самые интересные события в городе', 'с девушкой', 'с парнем', '6+'], source='kudago'

RAG оценка:   2%|▏         | 2/100 [00:03<02:32,  1.56s/it]

...

RAG оценка:  92%|█████████▏| 92/100 [01:02<00:05,  1.50it/s]

events:  [Event(title='лекция «Мировая литература на экране. Интересное об известном»', owner='all', description='Уникальный кинолекторий предлагает зрителям погрузиться в мир великих произведений XIX—XX веков через призму киноискусства.', tags=['all', 'education', 'msk', 'бесплатно', 'обучение', 'книги', 'free', 'лекции и мастер-классы', 'культура и искусство', '0+'], source='kudago', country='Россия', location='философский центр «Идея», ул. Новорогожская, д. 5', date='2024-10-29 11:30', url='https://kudago.com/msk/event/lektsiya-moskovskoe-dolgoletie-mirovaya-literatura-na-ekrane-interesnoe-ob-izvestnom/', uuid='1ceadd92-3611-5546-83be-64e0e4d27ad2'), Event(title='Бесплатные лекции о живописи в библиотеке имени Чехова', owner='all', description='Библиотека имени Чехова приглашает любителей искусства на увлекательные лекции о живописи.', tags=['all', 'education', 'spb', 'бесплатно', 'обучение', 'творческие события', 'free', 'лекции и мастер-классы', 'культура и искусство', 'искусство'

RAG оценка:  92%|█████████▏| 92/100 [01:03<00:05,  1.46it/s]

events:  [Event(title='шоу «Кощей» в «Москвариуме»', owner='all', description='На уникальной водной сцене «Москвариума» покажут волшебную новогоднюю сказку для всей семьи. В шоу с участием морских животных выступят акробаты, гимнасты и солисты мюзиклов, а современные спецэффекты поразят воображение.', tags=['all', 'kids', 'msk', 'детям', 'новый год 2026', 'волшебство', 'новый год с детьми', 'животные', 'новогодние праздники', 'развлечения', 'новогодние ёлки и спектакли', 'шоу', '0+'], source='kudago', country='Россия', location='«Москвариум», просп. Мира, д. 119, стр. 23, м. Ботанический сад, ВДНХ', date='2024-12-14 00:00 - 2024-12-16 00:00', url='https://kudago.com/msk/event/shou-koschej/', uuid='78c8e677-2450-5412-803d-b8b1faae659c'), Event(title='концерт «Новый год с Юрием Башметом»', owner='all', description='Новогодний вечер с участием одного из самых узнаваемых дирижёров страны обещает стать незабываемым завершением года. Авторская программа, живая энергетика и мощный художествен

KeyError: 'В саду «Эрмитаж» будет представлен проект «Музыкальный театр в парке», который объединит праздничное представление под открытым небом по мотивам сказки «12 месяцев», художественную инсталляцию и тематические активности для посетителей разных возрастов. Посетители парка смогут увидеть яркое шоу «Волшебные подснежники», режиссером которого выступает Алексей Франдетти. Фигуры месяцев оживут и станут частью иммерсивного сюжета.'

In [20]:
evaluation_results

[{'query': 'Где можно услышать симфоническое исполнение песен Rammstein?',
  'ground_truth_id': 623,
  'retrieved_ids': [623,
   451,
   338,
   380,
   380,
   29,
   558,
   285,
   294,
   516,
   546,
   541,
   466,
   221,
   221],
  'num_retrieved': 15},
 {'query': 'Какое музыкальное шоу предлагает необычные аранжировки хитов немецкой группы?',
  'ground_truth_id': 623,
  'retrieved_ids': [623,
   29,
   380,
   380,
   362,
   362,
   284,
   522,
   517,
   567,
   349,
   517,
   411,
   434,
   224],
  'num_retrieved': 15},
 {'query': 'Где можно весело провести выходной с детьми на комедийном шоу?',
  'ground_truth_id': 409,
  'retrieved_ids': [409,
   409,
   275,
   319,
   275,
   275,
   57,
   344,
   389,
   530,
   588,
   545,
   346,
   389,
   478],
  'num_retrieved': 15},
 {'query': 'Есть ли в городе стендап-клубы с выступлениями для детей?',
  'ground_truth_id': 409,
  'retrieved_ids': [409,
   409,
   417,
   437,
   644,
   608,
   509,
   644,
   509,
   608,


In [14]:
description_to_id['Хиты легендарной немецкой команды ещё никогда не звучали столь грандиозно.']

623

In [49]:
find_id = 529
for k,v in description_to_id.items():
    if v == find_id:
        print(k)
        break


Вы одиноки и мечтаете найти ворую половинку? На увлекательных мероприятиях от компании FastLife вы сможете за один вечер завести массу новых знакомств, расширить круг общения и приятно провести время в компании интересных людей, стремящихся найти свою любовь.


In [47]:
evaluation_results

[{'query': 'Где можно услышать симфоническое исполнение песен Rammstein?',
  'ground_truth_id': 630,
  'retrieved_ids': [529,
   289,
   554,
   80,
   189,
   538,
   538,
   503,
   392,
   315,
   503,
   599,
   599,
   515,
   337],
  'num_retrieved': 15},
 {'query': 'Какое музыкальное шоу предлагает необычные аранжировки хитов немецкой группы?',
  'ground_truth_id': 630,
  'retrieved_ids': [469,
   469,
   483,
   538,
   538,
   529,
   256,
   518,
   94,
   561,
   518,
   542,
   542,
   289,
   503],
  'num_retrieved': 15},
 {'query': 'Где можно весело провести выходной с детьми на комедийном шоу?',
  'ground_truth_id': 271,
  'retrieved_ids': [94,
   469,
   469,
   280,
   280,
   280,
   385,
   561,
   385,
   80,
   529,
   483,
   367,
   367,
   542],
  'num_retrieved': 15},
 {'query': 'Есть ли в городе стендап-клубы с выступлениями для детей?',
  'ground_truth_id': 271,
  'retrieved_ids': [80,
   433,
   540,
   515,
   515,
   499,
   499,
   7,
   56,
   262,
   35

## 7. Метрики RAG


In [21]:
def recall_at_k(retrieved_ids: List[int], relevant_id: int, k: int) -> float:
    """Recall@K - найден ли релевантный документ в топ-K."""
    return 1.0 if relevant_id in retrieved_ids[:k] else 0.0


def precision_at_k(retrieved_ids: List[int], relevant_id: int, k: int) -> float:
    """Precision@K - доля релевантных среди топ-K."""
    top_k = retrieved_ids[:k]
    relevant_count = 1 if relevant_id in top_k else 0
    return relevant_count / k if k > 0 else 0.0


def reciprocal_rank(retrieved_ids: List[int], relevant_id: int) -> float:
    """Reciprocal Rank - 1/позиция первого релевантного."""
    try:
        rank = retrieved_ids.index(relevant_id) + 1
        return 1.0 / rank
    except ValueError:
        return 0.0


def dcg_at_k(retrieved_ids: List[int], relevant_id: int, k: int) -> float:
    """DCG@K для бинарной релевантности."""
    dcg = 0.0
    for i, doc_id in enumerate(retrieved_ids[:k], 1):
        if doc_id == relevant_id:
            dcg += 1.0 / np.log2(i + 1)
    return dcg


def ndcg_at_k(retrieved_ids: List[int], relevant_id: int, k: int) -> float:
    """NDCG@K - нормализованный DCG."""
    dcg = dcg_at_k(retrieved_ids, relevant_id, k)
    # Идеальный DCG: релевантный документ на 1й позиции
    idcg = 1.0 / np.log2(2)  # = 1.0
    return dcg / idcg if idcg > 0 else 0.0


def hit_at_k(retrieved_ids: List[int], relevant_id: int, k: int) -> float:
    """Hit@K - есть ли попадание в топ-K."""
    return 1.0 if relevant_id in retrieved_ids[:k] else 0.0


print("Метрики загружены ✓")


Метрики загружены ✓


In [22]:
# Вычисляем метрики для каждого запроса
metrics_per_query = []

for res in evaluation_results:
    retrieved = res["retrieved_ids"]
    relevant = res["ground_truth_id"]
    
    query_metrics = {
        "query": res["query"][:50] + "...",
        "ground_truth_id": relevant,
        "mrr": reciprocal_rank(retrieved, relevant),
    }
    
    for k in K_VALUES:
        query_metrics[f"recall@{k}"] = recall_at_k(retrieved, relevant, k)
        query_metrics[f"precision@{k}"] = precision_at_k(retrieved, relevant, k)
        query_metrics[f"ndcg@{k}"] = ndcg_at_k(retrieved, relevant, k)
        query_metrics[f"hit@{k}"] = hit_at_k(retrieved, relevant, k)
    
    metrics_per_query.append(query_metrics)

df_metrics = pd.DataFrame(metrics_per_query)
print(f"Метрики вычислены для {len(df_metrics)} запросов")
df_metrics.head(10)


Метрики вычислены для 92 запросов


,query,ground_truth_id,mrr,recall@1,precision@1,ndcg@1,hit@1,recall@3,precision@3,ndcg@3,hit@3,recall@5,precision@5,ndcg@5,hit@5,recall@10,precision@10,ndcg@10,hit@10
0,Где можно услышать симфоническое исполнение пе...,623,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.00000,1.0,1.0,0.2,1.000000,1.0,1.0,0.1,1.000000,1.0
1,Какое музыкальное шоу предлагает необычные ара...,623,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.00000,1.0,1.0,0.2,1.000000,1.0,1.0,0.1,1.000000,1.0
2,Где можно весело провести выходной с детьми на...,409,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.63093,1.0,1.0,0.2,1.630930,1.0,1.0,0.1,1.630930,1.0
3,Есть ли в городе стендап-клубы с выступлениями...,409,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.63093,1.0,1.0,0.2,1.630930,1.0,1.0,0.1,1.630930,1.0
4,Где в Москве можно посетить мастер-класс по ке...,493,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.63093,1.0,1.0,0.2,1.630930,1.0,1.0,0.1,1.630930,1.0
5,"Есть ли в городе студии, где можно попробовать...",493,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.63093,1.0,1.0,0.2,1.630930,1.0,1.0,0.1,1.630930,1.0
6,Где можно послушать музыку из фильма «Аватар» ...,477,1.000000,1.0,1.0,1.0,1.0,1.0,0.333333,1.00000,1.0,1.0,0.2,1.000000,1.0,1.0,0.1,1.000000,1.0
7,Какие концерты проходят в Планетарии 1 в ближа...,477,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,1.0,0.2,0.430677,1.0,1.0,0.1,0.430677,1.0
8,Где можно увидеть выставку с куклами известных...,87,0.500000,0.0,0.0,0.0,0.0,1.0,0.333333,0.63093,1.0,1.0,0.2,0.630930,1.0,1.0,0.1,0.630930,1.0
9,"Какие художественные проекты можно посетить, ч...",87,0.333333,0.0,0.0,0.0,0.0,1.0,0.333333,0.50000,1.0,1.0,0.2,0.500000,1.0,1.0,0.1,0.500000,1.0


In [23]:
# Вычисляем средние метрики
numeric_cols = [col for col in df_metrics.columns if col not in ["query", "ground_truth_id"]]
mean_metrics = df_metrics[numeric_cols].mean()

print("=" * 60)
print("СВОДНЫЕ РЕЗУЛЬТАТЫ RAG EVALUATION")
print("=" * 60)

print(f"\n📊 MRR (Mean Reciprocal Rank): {mean_metrics['mrr']:.4f}")

print(f"\n📈 Recall@K:")
for k in K_VALUES:
    print(f"   Recall@{k}: {mean_metrics[f'recall@{k}']:.4f}")

print(f"\n🎯 Precision@K:")
for k in K_VALUES:
    print(f"   Precision@{k}: {mean_metrics[f'precision@{k}']:.4f}")

print(f"\n📉 NDCG@K:")
for k in K_VALUES:
    print(f"   NDCG@{k}: {mean_metrics[f'ndcg@{k}']:.4f}")

print(f"\n✅ Hit Rate@K:")
for k in K_VALUES:
    print(f"   Hit@{k}: {mean_metrics[f'hit@{k}']:.4f}")

print("\n" + "=" * 60)


СВОДНЫЕ РЕЗУЛЬТАТЫ RAG EVALUATION

📊 MRR (Mean Reciprocal Rank): 0.6862

📈 Recall@K:
   Recall@1: 0.5978
   Recall@3: 0.7609
   Recall@5: 0.8043
   Recall@10: 0.8804

🎯 Precision@K:
   Precision@1: 0.5978
   Precision@3: 0.2536
   Precision@5: 0.1609
   Precision@10: 0.0880

📉 NDCG@K:
   NDCG@1: 0.5978
   NDCG@3: 0.7688
   NDCG@5: 0.7917
   NDCG@10: 0.8163

✅ Hit Rate@K:
   Hit@1: 0.5978
   Hit@3: 0.7609
   Hit@5: 0.8043
   Hit@10: 0.8804



In [24]:
# Красивая таблица сводных метрик
summary_data = {
    "Метрика": [],
    "Значение": [],
}

summary_data["Метрика"].append("MRR")
summary_data["Значение"].append(f"{mean_metrics['mrr']:.4f}")

for k in K_VALUES:
    summary_data["Метрика"].append(f"Recall@{k}")
    summary_data["Значение"].append(f"{mean_metrics[f'recall@{k}']:.4f}")

for k in K_VALUES:
    summary_data["Метрика"].append(f"Precision@{k}")
    summary_data["Значение"].append(f"{mean_metrics[f'precision@{k}']:.4f}")

for k in K_VALUES:
    summary_data["Метрика"].append(f"NDCG@{k}")
    summary_data["Значение"].append(f"{mean_metrics[f'ndcg@{k}']:.4f}")

for k in K_VALUES:
    summary_data["Метрика"].append(f"Hit@{k}")
    summary_data["Значение"].append(f"{mean_metrics[f'hit@{k}']:.4f}")

df_summary = pd.DataFrame(summary_data)
df_summary


,Метрика,Значение
0,MRR,0.6862
1,Recall@1,0.5978
2,Recall@3,0.7609
3,Recall@5,0.8043
4,Recall@10,0.8804
5,Precision@1,0.5978
6,Precision@3,0.2536
7,Precision@5,0.1609
8,Precision@10,0.0880
9,NDCG@1,0.5978


In [ ]:
# Сохраняем результаты
results_file = testing_data_dir / "rag_evaluation_results.json"
results_data = {
    "num_queries": len(df_queries),
    "metrics": {
        "mrr": float(mean_metrics["mrr"]),
        "recall": {str(k): float(mean_metrics[f"recall@{k}"]) for k in K_VALUES},
        "precision": {str(k): float(mean_metrics[f"precision@{k}"]) for k in K_VALUES},
        "ndcg": {str(k): float(mean_metrics[f"ndcg@{k}"]) for k in K_VALUES},
        "hit_rate": {str(k): float(mean_metrics[f"hit@{k}"]) for k in K_VALUES},
    },
}

with open(results_file, "w", encoding="utf-8") as f:
    json.dump(results_data, f, ensure_ascii=False, indent=2)

print(f"✓ Результаты сохранены: {results_file}")


## 9. Интерпретация результатов

### Метрики:

- **MRR (Mean Reciprocal Rank)**: Среднее 1/позиция первого релевантного документа. Чем ближе к 1, тем лучше.
- **Recall@K**: Доля найденных релевантных документов в топ-K. Показывает полноту.
- **Precision@K**: Доля релевантных среди топ-K. Показывает точность.
- **NDCG@K**: Нормализованный DCG, учитывает позицию релевантного документа.
- **Hit@K**: Есть ли хотя бы один релевантный в топ-K.

### Рекомендации по улучшению:

- Низкий Recall → увеличить лимит результатов или улучшить эмбеддинги
- Низкий Precision → добавить фильтрацию или re-ranking
- Низкий MRR → релевантные документы не на первых позициях, нужен re-ranker


In [ ]:
# Визуализация
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Recall@K
    ax1 = axes[0, 0]
    recall_values = [mean_metrics[f"recall@{k}"] for k in K_VALUES]
    ax1.bar([str(k) for k in K_VALUES], recall_values, color='steelblue')
    ax1.set_xlabel('K')
    ax1.set_ylabel('Recall')
    ax1.set_title('Recall@K')
    ax1.set_ylim(0, 1)
    for i, v in enumerate(recall_values):
        ax1.text(i, v + 0.02, f'{v:.3f}', ha='center')
    
    # Precision@K
    ax2 = axes[0, 1]
    precision_values = [mean_metrics[f"precision@{k}"] for k in K_VALUES]
    ax2.bar([str(k) for k in K_VALUES], precision_values, color='coral')
    ax2.set_xlabel('K')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision@K')
    ax2.set_ylim(0, 1)
    for i, v in enumerate(precision_values):
        ax2.text(i, v + 0.02, f'{v:.3f}', ha='center')
    
    # NDCG@K
    ax3 = axes[1, 0]
    ndcg_values = [mean_metrics[f"ndcg@{k}"] for k in K_VALUES]
    ax3.bar([str(k) for k in K_VALUES], ndcg_values, color='seagreen')
    ax3.set_xlabel('K')
    ax3.set_ylabel('NDCG')
    ax3.set_title('NDCG@K')
    ax3.set_ylim(0, 1)
    for i, v in enumerate(ndcg_values):
        ax3.text(i, v + 0.02, f'{v:.3f}', ha='center')
    
    # Hit@K
    ax4 = axes[1, 1]
    hit_values = [mean_metrics[f"hit@{k}"] for k in K_VALUES]
    ax4.bar([str(k) for k in K_VALUES], hit_values, color='orchid')
    ax4.set_xlabel('K')
    ax4.set_ylabel('Hit Rate')
    ax4.set_title('Hit Rate@K')
    ax4.set_ylim(0, 1)
    for i, v in enumerate(hit_values):
        ax4.text(i, v + 0.02, f'{v:.3f}', ha='center')
    
    plt.tight_layout()
    plt.savefig('testing_data/rag_metrics.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ График сохранён: testing_data/rag_metrics.png")
    
except ImportError:
    print("matplotlib не установлен")
